In [1]:
# pick a hmm classifier and run through features,
# two different ways to analyse accuracy, confusion matrix per sample or combined model

In [2]:
import sys
sys.path.append("/Users/david/Documents/code/fhmm/v1")

In [3]:

import numpy as np
import os
import os.path
from pathlib import Path
import matplotlib.pyplot as plt
from importlib import reload
import src.misc_davidjames9610.fe_methods as fe
import src.misc_davidjames9610.proc_methods as pm
import importlib
from hmmlearn.hmm import GaussianHMM, GMMHMM

import src.classifiers_davidjames9610.test_a.config as base_config
reload(base_config)
config = importlib.import_module(base_config.config_location)
reload(config)
print(config.basedir)

/Users/david/Documents/code/fhmm/v1/src/classifiers_davidjames9610/test_a/speech_commands


In [4]:
import pickle
import src.misc_davidjames9610.utils as utils
sls = pickle.load(open(config.samples_labels,'rb'))
features = utils.folder_pickles_to_dict(config.basedir + '/features', 'mfcc')
# features = pickle.load(open(config.features,'rb'))

loading mfcc_RealNoise_UNDER_WATER_DRONE_SNR40.pickle
loading mfcc_GaussNoise_SNR60.pickle
loading mfcc_RealNoise_UNDER_WATER_DRONE_SNR60.pickle
loading mfcc_GaussNoise_SNR40.pickle
loading mfcc_None_None.pickle
loading mfcc_RealNoise_UNDER_WATER_DRONE_SNR20.pickle
loading mfcc_GaussNoise_SNR10.pickle
loading mfcc_RealNoise_UNDER_WATER_DRONE_SNR10.pickle
loading mfcc_GaussNoise_SNR20.pickle


In [5]:
print([key for key in features])

['mfcc_GaussNoise_SNR10', 'mfcc_GaussNoise_SNR20', 'mfcc_GaussNoise_SNR40', 'mfcc_GaussNoise_SNR60', 'mfcc_None_None', 'mfcc_RealNoise_UNDER_WATER_DRONE_SNR10', 'mfcc_RealNoise_UNDER_WATER_DRONE_SNR20', 'mfcc_RealNoise_UNDER_WATER_DRONE_SNR40', 'mfcc_RealNoise_UNDER_WATER_DRONE_SNR60']


In [6]:
# config
number_of_iterations = 1
n_components = 8

In [13]:
classifiers = {
    # {
    # 'type': 'GaussianHMM', # 'fhmm', 'GMMHMM',
    # 'features': ['lp', 'lp_GaussNoise_SNR60', 'lp_GaussNoise_SNR40', 'lp_GaussNoise_SNR20', 'lp_GaussNoise_SNR10', 'lp_RealNoise_UNDER_WATER_DRONE_SNR60', 'lp_RealNoise_UNDER_WATER_DRONE_SNR40', 'lp_RealNoise_UNDER_WATER_DRONE_SNR20', 'lp_RealNoise_UNDER_WATER_DRONE_SNR10',],
    # 'trained_classifiers': {}
    # },
    # {
    # 'type': 'fhmm', # 'fhmm', 'GMMHMM',
    # 'features': ['lp_GaussNoise_SNR60', 'lp_GaussNoise_SNR40', 'lp_GaussNoise_SNR20', 'lp_GaussNoise_SNR10', 'lp_RealNoise_UNDER_WATER_DRONE_SNR60', 'lp_RealNoise_UNDER_WATER_DRONE_SNR40', 'lp_RealNoise_UNDER_WATER_DRONE_SNR20', 'lp_RealNoise_UNDER_WATER_DRONE_SNR10'],
    # 'trained_classifiers': {}
    # },
    'GMMHMM': {
    'type': 'GMMHMM', # 'fhmm', 'GMMHMM',
    'features': [key for key in features], # ['mfcc_None_None', 'mfcc_GaussNoise_SNR60', 'mfcc_GaussNoise_SNR40', 'mfcc_GaussNoise_SNR20', 'mfcc_GaussNoise_SNR10'],
    'trained_classifiers': {}
    },
}

In [14]:
import src.fhmm_davidjames9610.fhmm as fhmm
reload(fhmm)
# classifiers = {} # one for feature
cv_index = 0 # TODO expand later

for classifier_key in classifiers:

    classifier = classifiers[classifier_key]

    all_features_for_classifier = classifier['features']
    classifier_type = classifier['type']

    print('For classifier:', classifier_type)

    for feature_key in all_features_for_classifier:
        print('  Training on:', feature_key)

        classifier['trained_classifiers'][feature_key] = {}

        curr_features = features[feature_key]['train_features'][cv_index]
        curr_labels = features[feature_key]['train_label'][cv_index]
        noise_features = None

        if classifier_type == 'fhmm':
            noise_features = features[feature_key]['noise_features'][cv_index]
            noise_features = np.array([item for sublist in noise_features for item in sublist])

        for speaker in sls['labels_set']:

            print('training for speaker: ', speaker)

            speaker_num = sls['label_to_num'][speaker]
            mask = (curr_labels == speaker_num)
            mask = mask.tolist()

            speaker_features = [array for array, flag in zip(curr_features, mask) if flag]
            speaker_features = np.array([item for sublist in speaker_features for item in sublist]) # TODO use lengths instead of flattening ?

            curr_classifier = None
            if classifier_type == 'fhmm':
                # FHMM
                curr_classifier = fhmm.FHMM(n_components_a=n_components, n_components_b=2)
                curr_classifier.fit(speaker_features, noise_features)
            elif classifier_type == 'GaussianHMM':
                # GAUSSIAN HMM
                curr_classifier = GaussianHMM(n_components=n_components, covariance_type='diag')
                curr_classifier.fit(speaker_features)
            elif classifier_type == 'GMMHMM':
                # GAUSSIAN HMM
                curr_classifier = GMMHMM(n_components=n_components, covariance_type='diag', n_mix=2)
                curr_classifier.fit(speaker_features)

            classifier['trained_classifiers'][feature_key][speaker] = curr_classifier

# use_fhmm = False
# print('training for feature type: ', feature_key)

# if feature_key.__contains__('Noise') and feature_key.__contains__('lp'):
#     print('using fhmm!')
#     use_fhmm = True

# classifiers[feature_key] = {}
# curr_features = features[feature_key]['train_features'][cv_index]
# curr_labels = features[feature_key]['train_label'][cv_index]

# if use_fhmm:
#     noise_features = features[feature_key]['noise_features'][cv_index]
#     noise_features = np.array([item for sublist in noise_features for item in sublist])
# print('score: ', curr_classifier.score(speaker_features))



For classifier: GMMHMM
  Training on: mfcc_GaussNoise_SNR10
training for speaker:  a2aca1e72
training for speaker:  a893705bb
training for speaker:  ab5cf6ea8
training for speaker:  ab66f4f93
training for speaker:  ac1d39ce8
training for speaker:  acce7416f
training for speaker:  addedba85
  Training on: mfcc_GaussNoise_SNR20
training for speaker:  a2aca1e72
training for speaker:  a893705bb
training for speaker:  ab5cf6ea8
training for speaker:  ab66f4f93
training for speaker:  ac1d39ce8
training for speaker:  acce7416f
training for speaker:  addedba85
  Training on: mfcc_GaussNoise_SNR40
training for speaker:  a2aca1e72
training for speaker:  a893705bb
training for speaker:  ab5cf6ea8
training for speaker:  ab66f4f93
training for speaker:  ac1d39ce8
training for speaker:  acce7416f
training for speaker:  addedba85
  Training on: mfcc_GaussNoise_SNR60
training for speaker:  a2aca1e72
training for speaker:  a893705bb
training for speaker:  ab5cf6ea8
training for speaker:  ab66f4f93
trai

In [15]:
utils.dict_to_folder_pickles(config.basedir + '/classifiers', classifiers)

Directory '/Users/david/Documents/code/fhmm/v1/src/classifiers_davidjames9610/test_a/speech_commands/classifiers' already exists, removing old files:  False
saving / updating  GMMHMM


In [11]:
import pickle
pickle.dump(classifiers, open(config.classifiers,'wb'))